# Quadratic-Penalty Steepest-Descent Sudoku Solver (Jupyter Version)

This notebook reworks the original `sudokusolver.py` script【turn3file0】 into a **continuous optimization** Sudoku solver
using the 729-variable formulation discussed in our optimization course:

- Variables: $x_{ijk} \in \mathbb{R}_{\ge 0}$ for $i,j,k \in \{1,\dots,9\}$.
- Constraints: nonnegativity, clues, and the standard Sudoku row/column/box equalities.
- Objective: minimize
  $$
  f(x) = \sum_{i=1}^9 \sum_{j=1}^9 \sum_{k=1}^9 x_{ijk},
  $$
  subject to those constraints, approximated by a **quadratic penalty method**:
  $$
  P_2(x,\rho) = f(x) + \frac{\rho}{2}\,\lVert G(x) \rVert_2^2,
  $$
  where $G(x)=0$ collects all Sudoku equalities.

We solve $\min_x P_2(x,\rho)$ for an increasing sequence of penalty parameters $\rho$, using
**steepest descent with backtracking line search**, and then decode the resulting $x$ to an integer
Sudoku grid.

This notebook:

1. Parses `sudoku.txt` (80 puzzles at 4 difficulty levels) into NumPy arrays.
2. Implements the quadratic-penalty steepest-descent solver.
3. Solves all puzzles and reports the success rate.


In [1]:
import numpy as np
import re
from pathlib import Path

## 1. Parsing `sudoku.txt`

The file `sudoku.txt` is an R-style dump of 80 Sudoku puzzles at four difficulty levels.
Structure (schematically):

- Top-level markers: `[[1]]`, `[[2]]`, `[[3]]`, `[[4]]` for difficulty levels.
- Per-puzzle headers: `[[d]][[p]]` for difficulty `d` and puzzle index `p` (1 to 20).
- Then an R matrix header line: `[,1] [,2] ... [,9]`.
- Then 9 row lines of the form:
  ```
   [1,]    2    4    8    9    7    3    1    6    5
  ```
  where the first token encodes the row index and is ignored; the remaining 9 tokens are integers
  in `{0,...,9}`, with `0` meaning an empty cell.

We parse this into a dictionary:
```python
puzzles_by_difficulty = {1: [grid1, ..., grid20],
                         2: [...],
                         3: [...],
                         4: [...]}
```
where each `grid` is a `9×9` NumPy array of integers.


In [2]:
def parse_sudoku_txt(path):
    """Parse sudoku.txt into a dict {difficulty: [9x9 numpy arrays]}.

    The file is an R-like dump with entries like
    [[1]]
    [[1]][[1]]
      [,1] [,2] ... [,9]
     [1,]  ... 9 ints ...
    """
    text = Path(path).read_text().splitlines()

    puzzles_by_diff = {1: [], 2: [], 3: [], 4: []}
    current_diff = None
    current_grid = None
    current_rows = []

    # regex for headers of the form [[d]][[p]]
    header_re = re.compile(r"^\[\[(\d+)\]\]\[\[(\d+)\]\]$")

    for line in text:
        s = line.strip()
        if not s:
            continue

        # Detect per-puzzle header [[d]][[p]]
        m = header_re.match(s)
        if m:
            diff = int(m.group(1))
            # Start a new puzzle
            current_diff = diff
            current_grid = []
            current_rows = []
            continue

        # Skip matrix column header lines like "[,1] [,2] ..."
        if s.startswith("[,1]"):
            continue

        # Row lines start with something like "[1,]" then 9 integers
        if s.startswith("[") or s.startswith("[1,") or s.startswith("[2,") or s.startswith("[3,"):
            # Generic: first token is like "[1,]"; ignore it
            tokens = s.split()
            if tokens[0].startswith("["):
                tokens = tokens[1:]
            # Now tokens should be 9 integers
            if len(tokens) >= 9:
                row_vals = [int(t) for t in tokens[:9]]
                current_rows.append(row_vals)

                # When we have 9 rows, store the grid
                if len(current_rows) == 9:
                    grid = np.array(current_rows, dtype=int)
                    if current_diff is None:
                        raise ValueError("Found grid rows before difficulty header.")
                    puzzles_by_diff[current_diff].append(grid)
                    current_rows = []
            continue

    # Basic sanity checks
    for d in puzzles_by_diff:
        if len(puzzles_by_diff[d]) == 0:
            print(f"Warning: difficulty {d} has no puzzles parsed.")
    return puzzles_by_diff


# Parse sudoku.txt (assumed in the same directory as this notebook)
SUDOKU_PATH = "sudoku.txt"

puzzles_by_diff = parse_sudoku_txt(SUDOKU_PATH)
for d in sorted(puzzles_by_diff):
    print(f"Difficulty {d}: {len(puzzles_by_diff[d])} puzzles parsed.")

# Show the first puzzle of difficulty 1
if puzzles_by_diff[1]:
    print("\nDifficulty 1, puzzle 1 (0 = empty):")
    print(puzzles_by_diff[1][0])

Difficulty 1: 20 puzzles parsed.
Difficulty 2: 20 puzzles parsed.
Difficulty 3: 20 puzzles parsed.
Difficulty 4: 20 puzzles parsed.

Difficulty 1, puzzle 1 (0 = empty):
[[2 4 8 9 7 3 1 6 5]
 [7 0 3 1 0 0 0 0 2]
 [0 0 1 4 0 8 0 7 3]
 [0 2 0 7 3 0 0 5 1]
 [0 7 9 6 5 1 0 2 8]
 [1 0 6 0 8 0 7 0 0]
 [4 0 2 0 9 7 5 1 0]
 [9 0 7 0 0 6 0 8 4]
 [0 1 5 8 4 2 0 9 0]]


## 2. Continuous 729-variable formulation and constraint builder

We flatten indices $(i,j,k)$ with $i,j,k \in \{0,\dots,8\}$ (0-based) into a single index
$p \in \{0,\dots,728\}$ via
$$
p = i\cdot 81 + j\cdot 9 + k.
$$

We then build a list of linear equality constraints of the form
$$
g_\ell(x) = \sum_{p \in S_\ell} x_p - c_\ell = 0,
$$
for:

- **Cell constraints**: each cell $(i,j)$ has exactly one digit,
  $$\sum_k x_{ijk} = 1.$$
- **Row-digit constraints**: each digit $k$ appears exactly once in row $i$,
  $$\sum_j x_{ijk} = 1.$$
- **Column-digit constraints**: each digit $k$ appears exactly once in column $j$,
  $$\sum_i x_{ijk} = 1.$$
- **Box-digit constraints**: each digit $k$ appears exactly once in each 3×3 box.
- **Clue constraints**: if the puzzle has $a_{ij} = k+1$, then $x_{ijk} = 1$.

We store each constraint as a pair `(indices, c)` where `indices` is a list of flattened indices
and `c` is the right-hand side constant (usually 1; for clues, 1 as well).


In [3]:
def idx(i, j, k):
    """Flatten (i,j,k) with i,j,k ∈ {0,...,8} into p ∈ {0,...,728}."""
    return i * 81 + j * 9 + k


def build_constraints_for_grid(grid):
    """Build Sudoku equality constraints for a given 9x9 integer grid.

    grid[i,j] ∈ {0,...,9}, with 0 = empty.
    Returns a list of (indices, c) constraints.
    """
    constraints = []

    # Cell constraints: sum_k x_{ijk} = 1
    for i in range(9):
        for j in range(9):
            indices = [idx(i, j, k) for k in range(9)]
            constraints.append((indices, 1.0))

    # Row-digit constraints: sum_j x_{ijk} = 1
    for i in range(9):
        for k in range(9):
            indices = [idx(i, j, k) for j in range(9)]
            constraints.append((indices, 1.0))

    # Column-digit constraints: sum_i x_{ijk} = 1
    for j in range(9):
        for k in range(9):
            indices = [idx(i, j, k) for i in range(9)]
            constraints.append((indices, 1.0))

    # Box-digit constraints: sum_{(i,j) in box} x_{ijk} = 1
    for box_row in (0, 3, 6):
        for box_col in (0, 3, 6):
            for k in range(9):
                indices = []
                for i in range(box_row, box_row + 3):
                    for j in range(box_col, box_col + 3):
                        indices.append(idx(i, j, k))
                constraints.append((indices, 1.0))

    # Clue constraints: x_{ijk} = 1 when grid[i,j] = k+1
    for i in range(9):
        for j in range(9):
            val = grid[i, j]
            if val != 0:
                k = val - 1  # digit k+1
                indices = [idx(i, j, k)]
                constraints.append((indices, 1.0))

    return constraints


# Quick check on first puzzle
if puzzles_by_diff[1]:
    constraints_example = build_constraints_for_grid(puzzles_by_diff[1][0])
    print(f"Number of constraints for one puzzle: {len(constraints_example)}")

Number of constraints for one puzzle: 375


## 3. Quadratic penalty function, gradient, and projection

We define the **penalized objective**

$$
P_2(x, \rho) = f(x) + \frac{\rho}{2} \sum_\ell g_\ell(x)^2,
$$

where

- $f(x) = \sum_p x_p$ is the original objective (constant on the exact feasible set),
- each constraint is $g_\ell(x) = \sum_{p\in S_\ell} x_p - c_\ell$,
- $\rho > 0$ is the quadratic penalty parameter.

The gradient is

$$
\nabla P_2(x, \rho)
= \mathbf{1} + \rho \sum_\ell g_\ell(x)\, \nabla g_\ell(x),
$$

and since each $g_\ell$ is linear with coefficients 1 on its support $S_\ell$, this becomes

$$
[\nabla P_2(x, \rho)]_p
= 1 + \rho \sum_{\ell:\, p\in S_\ell} g_\ell(x).
$$

We implement this matrix-free, storing each constraint as `(indices, c)`.
We also enforce nonnegativity via projection:
$$
x \leftarrow \max(x, 0) \quad \text{(componentwise)}.
$$


In [4]:
def P2(x, rho, constraints):
    """Quadratic penalty objective P2(x, rho)."""
    f = x.sum()
    penalty = 0.0
    for indices, c in constraints:
        r = x[indices].sum() - c
        penalty += r * r
    return f + 0.5 * rho * penalty


def grad_P2(x, rho, constraints):
    """Gradient of P2(x, rho) (matrix-free)."""
    g = np.ones_like(x)  # gradient of f(x) = sum x
    for indices, c in constraints:
        r = x[indices].sum() - c
        if r != 0.0:
            g[indices] += rho * r
    return g


def constraint_violation_norm(x, constraints):
    """Return ||G(x)||_2, where G collects all g_ℓ(x)."""
    sumsq = 0.0
    for indices, c in constraints:
        r = x[indices].sum() - c
        sumsq += r * r
    return np.sqrt(sumsq)


def project_nonnegative(x):
    """Project x onto the nonnegative orthant (in-place)."""
    np.maximum(x, 0.0, out=x)
    return x

## 4. Steepest descent with quadratic penalty and Sudoku decoding

For a fixed penalty parameter $\rho$, we approximately minimize $P_2(x, \rho)$ with **steepest descent**:

$$
x^{t+1} = \Pi_{x\ge 0}\bigl(x^t - \alpha_t \nabla P_2(x^t, \rho)\bigr),
$$

with backtracking line search (Armijo condition) to pick $\alpha_t$.

We then use an **outer loop** over increasing $\rho$ values, reusing the minimizer from one
penalty parameter as the starting point for the next.

Finally, we decode the continuous $x$ into a Sudoku grid by, for each cell $(i,j)$, selecting
the digit $k$ with the largest $x_{ijk}$ (argmax). We then validate the resulting integer grid.


In [5]:
def steepest_descent_penalty(x0, rho, constraints,
                             max_iter_inner=5000,
                             tol_grad=1e-6,
                             tol_constr=1e-6):
    """Inner steepest-descent loop for a fixed penalty parameter rho."""
    x = x0.copy()
    for t in range(max_iter_inner):
        g = grad_P2(x, rho, constraints)
        grad_norm = np.linalg.norm(g)
        if grad_norm < tol_grad:
            break

        P_current = P2(x, rho, constraints)
        alpha = 1.0
        beta = 0.5
        c_armijo = 1e-4

        # Backtracking line search
        while True:
            x_trial = project_nonnegative(x - alpha * g)
            P_trial = P2(x_trial, rho, constraints)
            if P_trial <= P_current - c_armijo * alpha * grad_norm**2:
                break
            alpha *= beta
            if alpha < 1e-8:
                x_trial = x
                break

        x = x_trial
        constr_norm = constraint_violation_norm(x, constraints)
        if constr_norm < tol_constr:
            break

    return x


def is_near_integral(x, atol=1e-2):
    """Heuristic: check if x is close to 0/1 (not used for correctness, just diagnostics)."""
    # For each cell (i,j), check that one k is close to 1 and others close to 0
    for i in range(9):
        for j in range(9):
            indices = [idx(i, j, k) for k in range(9)]
            cell_vals = x[indices]
            k_star = np.argmax(cell_vals)
            if abs(cell_vals[k_star] - 1.0) > atol:
                return False
            if np.any(np.delete(cell_vals, k_star) > atol):
                return False
    return True


def decode_solution(x):
    """Decode continuous x into a 9x9 integer Sudoku grid via argmax over k."""
    grid = np.zeros((9, 9), dtype=int)
    for i in range(9):
        for j in range(9):
            indices = [idx(i, j, k) for k in range(9)]
            k_star = int(np.argmax(x[indices]))
            grid[i, j] = k_star + 1
    return grid


def is_valid_sudoku(grid, original_grid):
    """Check whether `grid` is a valid Sudoku solution respecting `original_grid` clues."""
    # Clues respected
    for i in range(9):
        for j in range(9):
            if original_grid[i, j] != 0 and grid[i, j] != original_grid[i, j]:
                return False

    # Rows
    target = set(range(1, 10))
    for i in range(9):
        if set(grid[i, :]) != target:
            return False

    # Columns
    for j in range(9):
        if set(grid[:, j]) != target:
            return False

    # Boxes
    for bi in (0, 3, 6):
        for bj in (0, 3, 6):
            block = grid[bi:bi+3, bj:bj+3].ravel()
            if set(block) != target:
                return False

    return True


def solve_sudoku_continuous(grid,
                            rho_list=(1.0, 50.0, 500.0, 5000.0, 20000.0),
                            max_iter_inner=3000):
    """Solve a single Sudoku puzzle via quadratic-penalty steepest descent.

    Returns (solution_grid, history), where history is a list of
    (rho, constr_norm, P2_value) triples.
    """
    constraints = build_constraints_for_grid(grid)

    # Initialize x uniformly in each cell
    x = np.full(9 * 9 * 9, 1.0 / 9.0)

    history = []

    for rho in rho_list:
        x = steepest_descent_penalty(
            x0=x,
            rho=rho,
            constraints=constraints,
            max_iter_inner=max_iter_inner,
            tol_grad=1e-6,
            tol_constr=1e-6,
        )
        constr_norm = constraint_violation_norm(x, constraints)
        P_val = P2(x, rho, constraints)
        history.append((rho, constr_norm, P_val))

    solution_grid = decode_solution(x)
    return solution_grid, history

## 5. Solving all puzzles from `sudoku.txt`

We now run the continuous quadratic-penalty solver on each of the 80 puzzles and report, per
difficulty level:

- Whether the solution is valid.
- Final constraint violation norm and penalized objective value.
- A small snippet of the solution for inspection.

This is primarily pedagogical: it illustrates how the quadratic penalty method behaves on a
discrete/combinatorial problem, rather than being a production-grade Sudoku solver.


In [ ]:
def solve_all_puzzles(puzzles_by_diff,
                        rho_list=(1.0, 50.0, 500.0, 5000.0, 20000.0),
                        max_iter_inner=3000):
    results = {}
    for d in sorted(puzzles_by_diff):
        puzzles = puzzles_by_diff[d]
        print(f"\n=== Difficulty {d} ===")
        success_count = 0
        total = len(puzzles)
        for idx_p, grid in enumerate(puzzles, start=1):
            print(f"Puzzle {idx_p}:")
            solution, history = solve_sudoku_continuous(
                grid,
                rho_list=rho_list,
                max_iter_inner=max_iter_inner,
            )
            valid = is_valid_sudoku(solution, grid)
            if valid:
                success_count += 1
            final_rho, final_constr, final_P2 = history[-1]
            print(f"  valid: {valid}; final constraint norm {final_constr:.2e}")
            print("  history (rho, constr_norm, P2):")
            for rho, cn, p2 in history:
                print(f"    rho={rho:g}, constr_norm={cn:.2e}, P2={p2:.3f}")
            # Show top-left 3x3 block as a quick check
            print("  top-left 3x3 block:")
            print(solution[:3, :3])
        results[d] = (success_count, total)
        rate = 100.0 * success_count / max(total, 1)
        print(f"\nSolved {success_count} / {total} puzzles for difficulty {d} (success rate {rate:.1f}%).")
    return results


# Run the solver on all puzzles (this may take some time)
results = solve_all_puzzles(puzzles_by_diff)


=== Difficulty 1 ===
Puzzle 1:
  valid: True; final constraint norm 2.11e-04
  history (rho, constr_norm, P2):
    rho=1, constr_norm=4.21e+00, P2=72.152
    rho=50, constr_norm=8.42e-02, P2=80.823
    rho=500, constr_norm=8.42e-03, P2=80.982
    rho=5000, constr_norm=8.42e-04, P2=80.998
    rho=20000, constr_norm=2.11e-04, P2=81.000
  top-left 3x3 block:
[[2 4 8]
 [7 9 3]
 [5 6 1]]
Puzzle 2:
